# Sprint 4

Aan het einde van de [vierde sprint](https://trello.com/b/IKwmMG6m/sprint-4-milestone-2-12-dec) moet de draft van het final report, en de git repository met de laatste source code worden opgeleverd. Voor deze sprint convertern wij de notebook naar een werkende python.py console applicatie.

### 1. Helper Functions

In [Sprint 2](http://localhost:8888/notebooks/Sprint%202.ipynb) en [Sprint 3](https://trello.com/b/EUxZhLuE/sprint-3-milestone-1-27-nov) hebben we methodes gemaakt om woorden te tellen en een intersectie tussen woordenlijsten uit te voeren, woorden te tellen, emails te lezen, etc... Deze methodes kunnen worden gebruikt bij de taken.

In [5]:
import csv
import nltk
import string
import pandas as pd
import numpy as np

# nltk.download('punkt')
# nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

def tokenize(body):
    tokens = word_tokenize(body)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('dutch'))
    words = [w for w in words if not w in stop_words]
#     stemmer = SnowballStemmer("dutch")
#     words = [stemmer.stem(word) for word in words]
    return words

def read_txt(filepath):
    with open(filepath, 'r') as file:
        body = file.read()
    return tokenize(body)     
    
def read_csv(filepath, delimiter=','):
    with open(filepath, 'r') as c:
        return [row for row in csv.reader(c, delimiter=delimiter)]

def generate_csv_from_array(filename, array):
    with open("res/" + filename + ".csv", 'w', newline='') as c:
        writer = csv.writer(c, delimiter=',')
        writer.writerow(array)
        
def intersection(array1, array2):
    """Returns a generator, use next(generator)"""
    return (i for i in array1 if i in array2)

In [6]:
# Quick testing dutch stopwords/stemming
tokenize('ik loop naar de maan, wij lopen naar de manen')

['loop', 'maan', 'wij', 'lopen', 'manen']

### 2. Categories (Classes) aanmaken op basis van email dump

    ~Stefan Schenk

De train data bevat labels voor iedere email. De labels kunnen als categorieen gekozen worden om emails te classificeren.

#### 2.1 Data inlezen

In [7]:
from collections import Counter

# Read all emails
rows = read_csv('./res/klachtendumpgemeente.csv', ';')
    
# Preview
print('Category:', rows[1][0])
print('x1:', rows[1][1])
print('x2:', rows[1][2])
print('x3:', rows[1][3]) 
print()

# Count categories (excluding first row)
categories = Counter()

rows[0][0] = 'Categorie'
for row in rows[1:]:
    row[0] = row[0].lower()
    categories[row[0]] += 1
    
# Categories
for c in sorted(categories):
    print(c, '=', categories[c])

Category: Belastingen
x1: Gisteren ontving ik, tot mijn verbazing, opnieuw een aanmaning betreffende de mij opgelegde aanslag 31449952 waarbij gedreigd wordt met allerlei maatregelen als ik de aanslag niet binnen enkele dagen betaal. <br />Enkele weken geleden (direct na ontvangst van de eerste aanslag) heb ik echter bezwaar<wbr />/beroep aangetekend tegen deze aanslag. Hier heb ik nog totaal niets over gehoord, zelfs geen ontvangstbevestiging. U zult begrijpen dat het mij vreemd overkomt dat ik nu wel gewoon deze aanmaning ontvang. Het lijkt erop dat de gemeente Amsterdam mijn bezwaar<wbr />/beroep gewoon naast zich neerlegt en lekker doorgaat met de invordering van de aanslag. <br /><br />Ik zou dan ook, voordat ik eventueel tot betaling overga, graag eerst vernemen hoe het precies staat met mijn bezwaar<wbr />/beroep. Hoe en wanneer wordt dit behandeld, wat is de uitslag ervan, etc. <br /><br /><br />Mede omdat u ook dreigt met het inschakelen van deurwaarders en allerlei bijbehoren

#### 2.2 Numpy proberen

In [175]:
# Paste data in np matrix
matrix = np.array(rows)
# Apply tokenize function
# np.apply_along_axis( tokenize, axis=1, arr=matrix )
matrix[1:,1:]

array([[ 'Gisteren ontving ik, tot mijn verbazing, opnieuw een aanmaning betreffende de mij opgelegde aanslag 31449952 waarbij gedreigd wordt met allerlei maatregelen als ik de aanslag niet binnen enkele dagen betaal. <br />Enkele weken geleden (direct na ontvangst van de eerste aanslag) heb ik echter bezwaar<wbr />/beroep aangetekend tegen deze aanslag. Hier heb ik nog totaal niets over gehoord, zelfs geen ontvangstbevestiging. U zult begrijpen dat het mij vreemd overkomt dat ik nu wel gewoon deze aanmaning ontvang. Het lijkt erop dat de gemeente Amsterdam mijn bezwaar<wbr />/beroep gewoon naast zich neerlegt en lekker doorgaat met de invordering van de aanslag. <br /><br />Ik zou dan ook, voordat ik eventueel tot betaling overga, graag eerst vernemen hoe het precies staat met mijn bezwaar<wbr />/beroep. Hoe en wanneer wordt dit behandeld, wat is de uitslag ervan, etc. <br /><br /><br />Mede omdat u ook dreigt met het inschakelen van deurwaarders en allerlei bijbehorende incassomaatre

#### 2.3 Pandas proberen

In [99]:
# Paste data in pandas dataframe
df = pd.DataFrame(rows[1:], columns=rows[0])
# Apply tokenize function
df[['Inhoud 1', 'Inhoud 2', 'Inhoud 3']] = df[['Inhoud 1', 'Inhoud 2', 'Inhoud 3']].applymap(tokenize)
df

,Categorie,Inhoud 1,Inhoud 2,Inhoud 3,
0,Belastingen,"[gister, ontving, verbaz, opnieuw, aanman, bet...",[],"[afhandel, lopend, bezwar, wbr, beroep]",
1,Belastingen,[biljetnummer],[],"[i, hav, recently, moved, into, my, hous, on, ...",
2,Openbare ruimte,"[stank, geluidsoverlast, bakkerij, onz, koopwo...","[her, gev, vergunn, bakkerij, woning]","[inspectie, kijk, bakkerij, uberhaupt, wel, ma...",
3,Belastingen,"[hallo, brief, ontvang, betal, gat, bepaald, a...",[],"[aangepast, word, br, span, restaurant, bedrij...",
4,Parkeren,"[vrijdag, bezoek, rijksmuseum, onz, auto, gepa...","[contact, gezocht, medewerker, garag, gaf, all...","[euro, overmak, war, recht, reken, nl, ingb, t...",
5,Overlast,"[geluidsoverlast, danc, event, terwijl, weesp,...",[],"[zorg, geluid, volgend, ker, zachter, uur, duu...",
6,Openbare ruimte,"[bewoner, amsterdam, binnenstad, zorgverlener,...","[all, medewerker, zegg, evenement, amsterdam, ...","[strikter, verlen, vergunn, evenement, beter, ...",
7,Openbare ruimte,"[nacht, oktober, vond, ade, plat, amsterdam, a...","[gemeent, heft, vergunn, afgegev, medegedeeld,...","[gemeent, voorkomt, ker, gebeurd, event, derge...",
8,Openbare ruimte,"[betreft, marathon, wbr, gedeelt, vondelpark, ...",[],"[renner, help, bv, luidspreker, park, stan, op...",
9,Parkeren,"[mevrouw, vindt, belach, parkeerboet, ontvang,...",[],"[wilt, bekeur, betaald, hoeft, fout, gemaakt, ...",


#### 2.4 Normale Python lists proberen

In [8]:
# WARNING! Intensive computation (not very optimized)
# It counts all possible relevant words for each category
category_words = Counter()

for c in categories:
    category_words[c] = Counter()
    
    print(c)
    
    for row in [row for row in rows[1:] if c == row[0]]:
        for col in [1,2,3]:
            body = tokenize(row[col])            
            for word in body:
                category_words[c][word] += 1 


overlast
parkeren
belastingen
basisinformatie
onderwijs, jeugd en zorg
14020
overig
werk en inkomen
stadsloket
openbare ruimte


In [24]:
# Store most common words in csv
amount = 50
for c in categories:
    top_words = []
    for i in category_words[c].most_common()[:amount]:
        top_words.append(i[0])
        
    generate_csv_from_array('/categories/' + c, top_words)

In [9]:
# Most common words
for c in categories:
    print(c, category_words[c].most_common(), '\n')

 [] 

overlast [('br', 59), ('geluidsoverlast', 25), ('wbr', 21), ('overlast', 20), ('uur', 15), ('muziek', 15), ('geluid', 14), ('nacht', 14), ('klacht', 13), ('gemeente', 12), ('deur', 11), ('nee', 11), ('gaat', 11), ('weer', 11), ('s', 10), ('vergunning', 9), ('woning', 9), ('werkzaamheden', 9), ('the', 9), ('amsterdam', 8)] 

parkeren [('br', 1422), ('auto', 362), ('euro', 318), ('parkeren', 315), ('we', 298), ('betalen', 285), ('wij', 274), ('wbr', 258), ('amsterdam', 246), ('cition', 229), ('gemeente', 211), ('uur', 207), ('nee', 202), ('pr', 192), ('terug', 187), ('betaald', 186), ('graag', 170), ('p', 169), ('r', 167), ('geparkeerd', 156)] 

belastingen [('br', 566), ('the', 142), ('gemeente', 125), ('aanslag', 119), ('i', 117), ('amsterdam', 98), ('brief', 76), ('ontvangen', 72), ('bedrag', 71), ('to', 62), ('wbr', 62), ('betalen', 59), ('adres', 52), ('mevrouw', 50), ('graag', 49), ('for', 49), ('a', 48), ('bezwaar', 48), ('belastingen', 46), ('wij', 43)] 

basisinformatie [(